In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, r"C:\Users\jleus\PycharmProjects\NBACNN\DataScraper")
import leagueRosters
import leagueScraper
import statScraper
import teamFinder
import teamRoster
import teamScraper
import urlConvertor
import playerScraper
import save_df

In [3]:
def csv_to_df(csv):
    return(pd.read_csv(csv))

In [110]:
def clean_stats(df):
    for x in range(len(df)):
        #Date
        date = df.loc[x, 'Date']
        date = list(date)
        newdate = ''
        for y in range(10):
            if date[y] != '-':
                newdate+=date[y]
        df.at[x, 'Date'] = float(newdate)

        #Age
        age = df.loc[x, 'Age']
        age = list(age)
        newage = ''
        for y in range(6):
            if age[y] != '-':
                newage+=age[y]
        df.at[x, 'Age'] = float(newage)

        #Location
        location = df.loc[x, 'Location']
        if location == '@':
            df.at[x, 'Location'] = 0
        else:
            df.at[x, 'Location'] = 1

        #GS
        gs = df.loc[x, 'GS']
        gs = list(gs)
        newgs = ''
        if len(gs) == 6:
            newgs = gs[-2]
        else:
            newgs = gs[-3] + gs[-2]
        newgs = float(newgs)
        if gs[0] == 'L':
            newgs = newgs * -1
        df.at[x, 'GS'] = newgs

        #Active
        if df.loc[x, 'Active'] == '0' or df.loc[x, 'Active'] == '1':
            df.at[x, 'Active'] = 1
        else:
            df.at[x, 'Active'] = 0

        #MP
        mp = df.loc[x, 'MP']
        mp = str(mp)
        mp = list(mp)
        newmp = ''
        for y in range(len(mp)):
            if mp[y] != ':':
                newmp+=mp[y]
        newmp = float(newmp)
        df.at[x, 'MP'] = newmp
    return df


In [131]:
def standerdize_stats(cleaned_df):
    df_means = cleaned_df.mean()
    df_stdevs = cleaned_df.std()
    normalized_df = cleaned_df
    columns = ['Age', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'PlayerID', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    for column in columns:
        normalized_df[column] = (normalized_df[column] - df_means[column])/df_stdevs[column]
    return normalized_df

In [132]:
csv = r"C:\Users\jleus\OneDrive\Documents\CSVS\Magic2002.csv"
df = standerdize_stats(simple_clean_stats(csv_to_df(csv)))

C:\Users\jleus\AppData\Local\Temp/ipykernel_16416/2179797318.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_means = cleaned_df.mean()
C:\Users\jleus\AppData\Local\Temp/ipykernel_16416/2179797318.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_stdevs = cleaned_df.std()


In [129]:
df

,Name,G,Date,Age,Tm,Location,Opp,GS,Active,MP,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,"Armstrong, Darrell",1.0,20011030.0,33130.0,ORL,1,TOR,1.932486,1,0.485222,...,-0.309621,-0.146468,0.977296,1.499418,0.249414,-0.142618,-0.456625,1.840785,-1.230664,1.998983
1,"Armstrong, Darrell",2.0,20011102.0,33133.0,ORL,0,IND,-0.811334,1,0.611054,...,-0.431472,-0.261715,1.198676,0.725145,-0.414744,-0.785867,0.913296,0.887884,1.351996,1.030131
2,"Armstrong, Darrell",3.0,20011103.0,33134.0,ORL,0,CLE,2.002841,1,0.148344,...,0.267141,-0.095247,1.933977,1.886554,0.326048,-0.785867,-0.466848,0.654953,-3.095919,1.358698
3,"Armstrong, Darrell",4.0,20011105.0,33136.0,ORL,1,SEA,-0.389208,1,1.740351,...,-0.057795,-0.056831,0.740102,0.338008,-0.414744,-0.785867,-0.323722,0.972586,0.634591,0.802661
4,"Armstrong, Darrell",5.0,20011106.0,33137.0,ORL,0,SAS,-1.163106,1,0.889794,...,-0.171523,-0.633066,0.945670,-0.629833,-0.414744,-0.136926,-0.855333,0.146739,1.065034,-0.191466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,"Williams, Monty",64.0,20020411.0,30185.0,ORL,1,NJN,-0.248499,1,-0.706991,...,-0.398979,-0.748313,-0.018918,-0.629833,-0.414744,1.491119,-0.262382,-0.403826,-0.154556,-0.823326
977,"Williams, Monty",65.0,20020412.0,30186.0,ORL,0,MIA,-0.459563,1,-0.863086,...,0.380869,-0.210494,-0.943974,4.248087,-0.414744,-0.785867,-1.223372,-0.763811,-1.948070,0.347722
978,"Williams, Monty",66.0,20020414.0,30188.0,ORL,1,PHI,-0.529917,1,-0.894942,...,-0.090289,0.122442,3.151572,1.383277,-0.414744,-0.154003,2.242324,-0.615582,0.132407,-0.334688
979,"Williams, Monty",67.0,20020416.0,30190.0,ORL,0,ATL,0.38469,1,1.176499,...,0.039686,0.007195,0.241995,0.376722,0.045058,-0.222313,0.504365,0.422021,-1.015443,0.701563
